In [1]:
#統整

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, IDF, CountVectorizer
from pyspark.sql.functions import rand 
from pyspark.sql.functions import lit
import pyspark.sql.functions as sparkf

if __name__ == "__main__":

    spark = SparkSession \
        .builder \
        .getOrCreate()


data_top_pre = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/top.csv", inferSchema=True, header=True)
data_top = data_top_pre.na.drop()
res1 = data_top.withColumn('id', sparkf.monotonically_increasing_id())
#res1.show()
#res1.count()

words_added = RegexTokenizer(inputCol="title", outputCol="Brand&Desc", pattern=r"\W+").transform(data_top)
Brand = words_added.select("Brand&Desc")
tf_added = CountVectorizer(inputCol="Brand&Desc", outputCol="tf_b").fit(Brand).transform(Brand)
tf_idf_added = IDF(inputCol="tf_b", outputCol="tf_idf_b_top").fit(tf_added).transform(tf_added)
#tf_idf_added.show(truncate=False)
res2 = tf_idf_added.withColumn('id', sparkf.monotonically_increasing_id())
#res2.show()


words_added2 = RegexTokenizer(inputCol="category", outputCol="sub_cates", pattern=r"\W+").transform(data_top)
sub_cate = words_added2.select("sub_cates") 
stopwordList = ["null", "Null", '', ' '] 
stopwordList.extend(StopWordsRemover().getStopWords())
stopwordList = list(set(stopwordList))
words_stop_words_removed = StopWordsRemover(inputCol="sub_cates", outputCol="sub_cates_removed", stopWords=stopwordList).transform(sub_cate)
sub_cates_rm = words_stop_words_removed.select("sub_cates_removed")
tf_added2= CountVectorizer(inputCol="sub_cates_removed", outputCol="tf_c").fit(sub_cates_rm).transform(sub_cates_rm)
tf_idf_added2 = IDF(inputCol="tf_c", outputCol="tf_idf_c_top").fit(tf_added2).transform(tf_added2)
#tf_idf_added2.show(truncate=False)
res3 = tf_idf_added2.withColumn('id', sparkf.monotonically_increasing_id())
#res3.show()

#join
ready_data_top = res1.join(res2, 'id').join(res3,'id').select('iid', 'tf_idf_b_top', 'tf_idf_c_top')
# ready_data_top.show(10)
# ready_data_top.printSchema()
